## Experiment Setup

### Random seed / PyTorch / CUDA related

In [1]:
import time
import datetime
import os
import sys
import itertools

# Use Google Colab
use_colab = True

# Is this notebook running on Colab?
# If so, then google.colab package (github.com/googlecolab/colabtools)
# should be available in this environment

# Previous version used importlib, but we could do the same thing with
# just attempting to import google.colab
try:
    from google.colab import drive
    colab_available = True
except:
    colab_available = False

if use_colab and colab_available:
    drive.mount('/content/drive')
    
    # If there's a package I need to install separately, do it here
    !pip install pyro-ppl

    # cd to the appropriate working directory under my Google Drive
    %cd 'drive/My Drive/Colab Notebooks/bayesian-dl-experiments'
    
    # List the directory contents
    !ls

# IPython reloading magic
%load_ext autoreload
%autoreload 2

# Random seeds
# Based on https://pytorch.org/docs/stable/notes/randomness.html
random_seed = 682

### Third party libraries (NumPy, PyTorch, Pyro)

In [2]:
# Third party libraries import
import numpy as np
import torch
import pyro
import matplotlib.pyplot as plt

# Print version information
print("NumPy Version: " + np.__version__)
print("PyTorch Version: " + torch.__version__)
print("Pyro Version: " + pyro.__version__)

NumPy Version: 1.17.2
PyTorch Version: 1.3.1
Pyro Version: 1.0.0


In [3]:
# More imports...
from torch import nn, optim
from torch.utils.data import random_split, DataLoader, RandomSampler
import torchvision
import torchvision.transforms as transforms
from pyro.infer import SVI, Trace_ELBO, HMC, MCMC

# Import model and dataset classes from ronald_bdl
from ronald_bdl import models, datasets
from ronald_bdl.models import utils

# pyplot setting
%matplotlib inline

# torch.device / CUDA Setup
use_cuda = True

if use_cuda and torch.cuda.is_available():
    torch_device = torch.device('cuda')

    torch.backends.cudnn.deterministic = True
            
    # Disable 'benchmark' mode
    # Note: https://discuss.pytorch.org/t/what-does-torch-backends-cudnn-benchmark-do/5936
    torch.backends.cudnn.benchmark = False
    use_pin_memory = True # Faster Host to GPU copies with page-locked memory

    # CUDA libraries version information
    print("CUDA Version: " + str(torch.version.cuda))
    print("cuDNN Version: " + str(torch.backends.cudnn.version()))
    print("CUDA Device Name: " + str(torch.cuda.get_device_name()))
    print("CUDA Capabilities: "+ str(torch.cuda.get_device_capability()))    
else:
    torch_device = torch.device('cpu')
    use_pin_memory = False

### Variable settings

#### Data prep

In [4]:
# Dataset to use
uci_dataset_name = 'bostonHousing'

# Set the proportion of the original dataset to be available as a whole
subset_proportions = [1]

# Proportion of the dataset (after getting the subset) to be used for training
train_prop = 0.9

# Number of dataset splits
n_splits = 10

#### NN settings

In [5]:
# Fully-Connected network hidden layers
network_hidden_dims = [25, 50, 100]
network_hidden_layers = [1, 3, 5]

# Dropout
dropout_rates = [0.1, 0.3, 0.5]

# Length-scale
length_scale_values = [1e-2]

# Model Precision
tau_values = [0.1, 0.15, 0.2]

#### Training setup

In [6]:
# Number of epochs
n_epoch = 4000

# Optimizer learning rate
learning_rate = 0.001

# Training data batch sizes
if uci_dataset_name in ('protein-tertiary-structure'): # Note: 45730 rows originally
    n_training_batch = 256
elif uci_dataset_name in ('yacht', 'bostonHousing'):
    n_training_batch = 32

# Number of test predictions (for each data point)
prediction_runs = [3, 10, 50, 100, 1000, 10000]

# Mean Squared Error for loss function to minimize
objective = nn.MSELoss()

# Test start time
test_start_time = datetime.datetime.today().strftime('%Y%m%d%H%M')

## Train the network

In [ ]:
for subset_prop, hidden_dim, n_hidden, dropout_rate, length_scale, tau in itertools.product(
    subset_proportions,
    network_hidden_dims, network_hidden_layers,
    dropout_rates, length_scale_values, tau_values,
):

    # Reset the random number generator for each method (to produce identical results)
    torch.manual_seed(random_seed)
    np.random.seed(random_seed)
    pyro.set_rng_seed(random_seed)

    """
    Results file storage
    """
 
    # Create directory to store results for the current test configuration
    test_results_path = os.path.join(
        './test_results',
        'number_of_test_predictions_1',
        uci_dataset_name,
        test_start_time,
        (
            str(subset_prop) 
            + '_' + str(hidden_dim) 
            + '_' + str(n_hidden) 
            + '_' + str(dropout_rate) 
            + '_' + str(length_scale)
            + '_' + str(tau)),
    )
    
    os.makedirs(test_results_path, exist_ok=True)
    
    test_results_rmse_mc_path = os.path.join(
        test_results_path,
        "rmse_mc.txt"
    )
    
    test_results_lls_mc_path = os.path.join(
        test_results_path,
        "lls_mc.txt"
    )

    # Prepare new subset of the original dataset
    subset = datasets.UCIDatasets(
        uci_dataset_name, root_dir='./datasets_files', 
        limit_size=subset_prop,
        transform=lambda X, mean, std: (X - mean) / std,
        target_transform=lambda y, mean, std: (y - mean) / std,
        download=True)

    # Determine sizes of training and testing set
    train_size = int(train_prop * len(subset))
    test_size = len(subset) - train_size
    
    # Print the size of the subset
    print("subset size = " + str((len(subset), subset.n_features)))
    print("training set size = %d" % train_size)
    print("test set size = %d" % test_size)
    
    train, test = random_split(subset, lengths=[train_size, test_size])

    train_loader = DataLoader(train, batch_size=n_training_batch, pin_memory=use_pin_memory)

    # Prepare network
    network = models.FCNet(
        input_dim=subset.n_features, 
        output_dim=subset.n_targets,
        hidden_dim=hidden_dim,
        n_hidden=n_hidden,
        dropout_rate=dropout_rate,
        dropout_type='bernoulli',
    )
    
    # Send the whole model to the selected torch.device
    network.to(torch_device)

    # Print the network structure
    print(network)
    
    # Model to train mode
    network.train()
    
    # Adam optimizer
    # https://pytorch.org/docs/stable/optim.html?highlight=adam#torch.optim.Adam

    # NOTE: Need to set L2 regularization from here
    reg_strength = utils.reg_strength(dropout_rate, length_scale, train_size, tau)
    print('reg_strength = ' + str(reg_strength))
        
    optimizer = optim.Adam(
        network.parameters(),
        lr=learning_rate,
        weight_decay=reg_strength, # L2 regularization
    )

    print()

    """
    Training
    """

    print(
        "Starting subset %f, n_hidden %d, hidden_dim %d, dropout_rate %f, length_scale %f, tau %f"
        % (subset_prop, n_hidden, hidden_dim, dropout_rate, length_scale, tau))

    # Record training start time (for this split)
    tic = time.time()

    for epoch in range(n_epoch): # loop over the dataset multiple times
        for i, data in enumerate(train_loader):
            # get the inputs; data is a list of [inputs, labels]
            inputs, targets = data

            # Store the batch to torch_device's memory
            inputs = inputs.to(torch_device)
            targets = targets.to(torch_device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = network(inputs)

            loss = objective(outputs, targets)
            loss.backward()

            optimizer.step()
            
    # Record training end time
    toc = time.time()

    # Report the final loss
    print("final loss = %f" % (loss.item()))

    """
    Testing
    """

    # Model to eval mode
    network.eval()

    # Get the test data
    inputs, targets = test.dataset[test.indices]

    # Store the batch to torch_device's memory
    inputs = inputs.to(torch_device)
    targets = targets.to(torch_device)

    for n_predictions in prediction_runs:
        print(str(n_predictions) + " test runs...")

        # Record testing start time (for this split)
        tic_testing = time.time()

        _, mean, var, metrics = network.predict_dist(
            inputs, n_predictions,
            y_test=targets, reg_strength=reg_strength, train_size=train_size)

        # Record testing end time
        toc_testing = time.time()

        # store additional metrics
        if len(metrics) > 0:

            for key, value in metrics.items():
                print(str(key) + " = " + str(value))

                if key == 'rmse_mc':
                    with open(test_results_rmse_mc_path, 'a+') as rmse_mc_file:
                        rmse_mc_file.write('%d %f \n' % (n_predictions, value))

                elif key == 'test_ll_mc':
                    with open(test_results_lls_mc_path, 'a+') as lls_mc_file:
                        lls_mc_file.write('%d %f \n' % (n_predictions, value))
            print()
            
    # Report the total training time
    print("training time = " + str(toc - tic) + " seconds")
    
    # Report the total testing time
    print("testing time (last run) = " + str(toc_testing - tic_testing) + " seconds")

    print()

Using downloaded and verified file: ./datasets_files/bostonHousing/housing.data
subset size = (506, 13)
training set size = 455
test set size = 51
FCNet(
  (input): ModuleDict(
    (dropout): Dropout(p=0.1, inplace=False)
    (linear): Linear(in_features=13, out_features=25, bias=True)
    (nonlinear): ReLU()
  )
  (hidden_layers): ModuleList(
    (0): ModuleDict(
      (dropout): Dropout(p=0.1, inplace=False)
      (linear): Linear(in_features=25, out_features=25, bias=True)
      (nonlinear): ReLU()
    )
  )
  (output): ModuleDict(
    (dropout): Dropout(p=0.1, inplace=False)
    (linear): Linear(in_features=25, out_features=1, bias=True)
  )
)
reg_strength = tensor(9.8901e-07)

Starting subset 1.000000, n_hidden 1, hidden_dim 25, dropout_rate 0.100000, length_scale 0.010000, tau 0.100000
final loss = 0.013793
3 test runs...
rmse_mc = tensor(0.3456)
rmse_non_mc = tensor(0.3400)
test_ll_mc = tensor(-2.0783)

10 test runs...
rmse_mc = tensor(0.3226)
rmse_non_mc = tensor(0.3400)
test_l

/Users/bseoh/Downloads/bayesian-dl-experiments-master/ronald_bdl/models/fcnet.py:183: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  kwargs['reg_strength'], dtype=torch.float)
/Users/bseoh/Downloads/bayesian-dl-experiments-master/ronald_bdl/models/fcnet.py:200: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  / (2 * train_size * reg_strength))


rmse_mc = tensor(0.3201)
rmse_non_mc = tensor(0.3400)
test_ll_mc = tensor(-2.0783)

training time = 166.89374589920044 seconds
testing time (last run) = 1.3497748374938965 seconds

Using downloaded and verified file: ./datasets_files/bostonHousing/housing.data
subset size = (506, 13)
training set size = 455
test set size = 51
FCNet(
  (input): ModuleDict(
    (dropout): Dropout(p=0.1, inplace=False)
    (linear): Linear(in_features=13, out_features=25, bias=True)
    (nonlinear): ReLU()
  )
  (hidden_layers): ModuleList(
    (0): ModuleDict(
      (dropout): Dropout(p=0.1, inplace=False)
      (linear): Linear(in_features=25, out_features=25, bias=True)
      (nonlinear): ReLU()
    )
  )
  (output): ModuleDict(
    (dropout): Dropout(p=0.1, inplace=False)
    (linear): Linear(in_features=25, out_features=1, bias=True)
  )
)
reg_strength = tensor(6.5934e-07)

Starting subset 1.000000, n_hidden 1, hidden_dim 25, dropout_rate 0.100000, length_scale 0.010000, tau 0.150000
final loss = 0.0

final loss = 0.166716
3 test runs...
rmse_mc = tensor(0.7534)
rmse_non_mc = tensor(0.7222)
test_ll_mc = tensor(-2.1020)

10 test runs...
rmse_mc = tensor(0.7338)
rmse_non_mc = tensor(0.7222)
test_ll_mc = tensor(-2.1016)

50 test runs...
rmse_mc = tensor(0.7203)
rmse_non_mc = tensor(0.7222)
test_ll_mc = tensor(-2.1004)

100 test runs...
rmse_mc = tensor(0.7184)
rmse_non_mc = tensor(0.7222)
test_ll_mc = tensor(-2.1007)

1000 test runs...
rmse_mc = tensor(0.7102)
rmse_non_mc = tensor(0.7222)
test_ll_mc = tensor(-2.1001)

10000 test runs...
rmse_mc = tensor(0.7083)
rmse_non_mc = tensor(0.7222)
test_ll_mc = tensor(-2.0999)

training time = 169.19795894622803 seconds
testing time (last run) = 1.4349689483642578 seconds

Using downloaded and verified file: ./datasets_files/bostonHousing/housing.data
subset size = (506, 13)
training set size = 455
test set size = 51
FCNet(
  (input): ModuleDict(
    (dropout): Dropout(p=0.5, inplace=False)
    (linear): Linear(in_features=13, out_features=25, 

final loss = 0.068448
3 test runs...
rmse_mc = tensor(0.5830)
rmse_non_mc = tensor(0.5534)
test_ll_mc = tensor(-1.7642)

10 test runs...
rmse_mc = tensor(0.5492)
rmse_non_mc = tensor(0.5534)
test_ll_mc = tensor(-1.7597)

50 test runs...
rmse_mc = tensor(0.5248)
rmse_non_mc = tensor(0.5534)
test_ll_mc = tensor(-1.7569)

100 test runs...
rmse_mc = tensor(0.5326)
rmse_non_mc = tensor(0.5534)
test_ll_mc = tensor(-1.7583)

1000 test runs...
rmse_mc = tensor(0.5332)
rmse_non_mc = tensor(0.5534)
test_ll_mc = tensor(-1.7583)

10000 test runs...
rmse_mc = tensor(0.5301)
rmse_non_mc = tensor(0.5534)
test_ll_mc = tensor(-1.7580)

training time = 200.01884198188782 seconds
testing time (last run) = 2.3796260356903076 seconds

Using downloaded and verified file: ./datasets_files/bostonHousing/housing.data
subset size = (506, 13)
training set size = 455
test set size = 51
FCNet(
  (input): ModuleDict(
    (dropout): Dropout(p=0.3, inplace=False)
    (linear): Linear(in_features=13, out_features=25, 

final loss = 0.282135
3 test runs...
rmse_mc = tensor(0.7634)
rmse_non_mc = tensor(0.7927)
test_ll_mc = tensor(-1.9129)

10 test runs...
rmse_mc = tensor(0.7950)
rmse_non_mc = tensor(0.7927)
test_ll_mc = tensor(-1.9164)

50 test runs...
rmse_mc = tensor(0.7793)
rmse_non_mc = tensor(0.7927)
test_ll_mc = tensor(-1.9155)

100 test runs...
rmse_mc = tensor(0.7782)
rmse_non_mc = tensor(0.7927)
test_ll_mc = tensor(-1.9155)

1000 test runs...
rmse_mc = tensor(0.7840)
rmse_non_mc = tensor(0.7927)
test_ll_mc = tensor(-1.9159)

10000 test runs...
rmse_mc = tensor(0.7820)
rmse_non_mc = tensor(0.7927)
test_ll_mc = tensor(-1.9158)

training time = 194.54095482826233 seconds
testing time (last run) = 2.2088170051574707 seconds

Using downloaded and verified file: ./datasets_files/bostonHousing/housing.data
subset size = (506, 13)
training set size = 455
test set size = 51
FCNet(
  (input): ModuleDict(
    (dropout): Dropout(p=0.5, inplace=False)
    (linear): Linear(in_features=13, out_features=25, 

final loss = 0.104699
3 test runs...
rmse_mc = tensor(0.5664)
rmse_non_mc = tensor(0.5564)
test_ll_mc = tensor(-1.7593)

10 test runs...
rmse_mc = tensor(0.5394)
rmse_non_mc = tensor(0.5564)
test_ll_mc = tensor(-1.7602)

50 test runs...
rmse_mc = tensor(0.5454)
rmse_non_mc = tensor(0.5564)
test_ll_mc = tensor(-1.7606)

100 test runs...
rmse_mc = tensor(0.5493)
rmse_non_mc = tensor(0.5564)
test_ll_mc = tensor(-1.7610)

1000 test runs...
rmse_mc = tensor(0.5439)
rmse_non_mc = tensor(0.5564)
test_ll_mc = tensor(-1.7606)

10000 test runs...
rmse_mc = tensor(0.5483)
rmse_non_mc = tensor(0.5564)
test_ll_mc = tensor(-1.7611)

training time = 228.0705442428589 seconds
testing time (last run) = 3.220449924468994 seconds

Using downloaded and verified file: ./datasets_files/bostonHousing/housing.data
subset size = (506, 13)
training set size = 455
test set size = 51
FCNet(
  (input): ModuleDict(
    (dropout): Dropout(p=0.3, inplace=False)
    (linear): Linear(in_features=13, out_features=25, bi

final loss = 0.267640
3 test runs...
rmse_mc = tensor(0.8236)
rmse_non_mc = tensor(0.8236)
test_ll_mc = tensor(-2.1042)

10 test runs...
rmse_mc = tensor(0.8230)
rmse_non_mc = tensor(0.8236)
test_ll_mc = tensor(-2.1041)

50 test runs...
rmse_mc = tensor(0.8236)
rmse_non_mc = tensor(0.8236)
test_ll_mc = tensor(-2.1042)

100 test runs...
rmse_mc = tensor(0.8236)
rmse_non_mc = tensor(0.8236)
test_ll_mc = tensor(-2.1042)

1000 test runs...
rmse_mc = tensor(0.8235)
rmse_non_mc = tensor(0.8236)
test_ll_mc = tensor(-2.1041)

10000 test runs...
rmse_mc = tensor(0.8236)
rmse_non_mc = tensor(0.8236)
test_ll_mc = tensor(-2.1041)

training time = 233.31720089912415 seconds
testing time (last run) = 3.526081085205078 seconds

Using downloaded and verified file: ./datasets_files/bostonHousing/housing.data
subset size = (506, 13)
training set size = 455
test set size = 51
FCNet(
  (input): ModuleDict(
    (dropout): Dropout(p=0.5, inplace=False)
    (linear): Linear(in_features=13, out_features=25, b

## Results visualization

In [ ]:
experiment_root_directory = os.path.join(
    './test_results',
    'number_of_test_predictions_1',
    uci_dataset_name,
    test_start_time,
)

for subset_prop, dropout_rate, length_scale, tau in itertools.product(
    subset_proportions, dropout_rates, length_scale_values, tau_values,
):
    for metric_name in ['lls_mc', 'rmse_mc']:
        figure_file_name = (
            str(subset_prop)
            + '_' + str(dropout_rate)
            + '_' + str(length_scale)
            + '_' + str(tau)
            + '_' + metric_name + '.png'
        )
        
        figure_title = (
            metric_name 
            + (' subset %f, dropout rate = %f, length_scale %f, tau %f' 
               % (subset_prop, dropout_rate, length_scale, tau))
        )

        fig, ax = plt.subplots(len(network_hidden_dims), len(network_hidden_layers))

        fig.tight_layout(pad=2, rect=[0, 0.00, 2, 2])    

        plt_subplot_current_row = 0
        
        for hidden_dim in network_hidden_dims:
            plt_subplot_current_column = 0

            for n_layer in network_hidden_layers:
                # Open the score file
                score_file_path = os.path.join(
                    experiment_root_directory,
                    (
                        str(subset_prop) 
                        + '_' + str(hidden_dim)
                        + '_' + str(n_layer) 
                        + '_' + str(dropout_rate) 
                        + '_' + str(length_scale)
                        + '_' + str(tau)
                    ),
                    metric_name + '.txt',
                )

                scores = np.loadtxt(score_file_path).T

                if metric_name == 'rmse_mc':
                    ax[plt_subplot_current_row, plt_subplot_current_column].set_ylim([0, 10])
                elif metric_name == 'lls_mc':
                    ax[plt_subplot_current_row, plt_subplot_current_column].set_ylim([-10, 0])

                ax[plt_subplot_current_row, plt_subplot_current_column].scatter(scores[0], scores[1])

                ax[plt_subplot_current_row, plt_subplot_current_column].set_title(str((hidden_dim, n_layer)))

                plt_subplot_current_column += 1

            plt_subplot_current_row += 1

        fig.suptitle(figure_title, y=2.05)
        
        plt.savefig(
            os.path.join(experiment_root_directory, figure_file_name),
            dpi=600,
            bbox_inches='tight',
        )
        
        plt.show()